![image](dependencies/open-science-prize.png)

## Higher Fidelity Graph States
In this notebook, we will prepare quantum circuits for a 7-qubit graph state and estimate the fidelity through stabilizer measurements using Qiskit. In this example, we use the CTMP method of error mitigation [1] and repeat the graph state measurement 16 times to find statistical error bars.

**To Do:
Modify the graph state preparation circuit or use your own methods of error mitigation to improve the graph state fidelity.**


[1] S. Bravyi, S. Sheldon, A. Kandala, D.C. McKay, J.M. Gambetta, Mitigating measurement errors in multi-qubit experiments, [arXiv:2006.14044](https://arxiv.org/abs/2006.14044) (2020).

## Imports

Begin by importing the necessary packages and defining the functions we will need for the stabilizer measurements.

In [1]:
### install Qiskit and other modules if you don't have them already
!pip install -r dependencies/requirements.txt --quiet

In [2]:
# Qiskit module
from qiskit import QuantumCircuit
import qiskit.circuit.library as circuit_library
import qiskit.quantum_info as qi
import qiskit.ignis.mitigation as mit

# Qiskit tools for noisy simulation
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.utils import insert_noise

# Qiskit tools for running and monitoring jobs
from qiskit import execute
from qiskit.tools.monitor import job_monitor

# Other imports
import numpy as np
from model import solve_graph_state
#import pickle
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

In order to run the circuits, first load the backend `ibmq_casablanca` from your account using the `IBMQ` provider. You will receive access to `ibm-q-community/ibmquantumawards/open-science` after registering for the Open Science Prize.

In [3]:
# Load IBMQ Account data
from qiskit import IBMQ
IBMQ.load_account()

# Get backend for experiment
provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science')
backend = provider.get_backend('ibmq_casablanca')
properties = backend.properties()

#nm = NoiseModel.from_backend(properties)
#sim_backend = QasmSimulator(method='density_matrix', basis_gates=['cx', 'id', 'rz', 'sx', 'x', 'barrier', 'kraus', 'roerror', 'snapshot'], noise_model=nm, max_parallel_experiments=-1)
#debug = True
#if debug:
#    backend = sim_backend

In [4]:
## Preparing graph states

## Preparing graph states

Here, we prepare the graph state circuit for 7-qubits using the `GraphState` function in Qiskit's circuit library. We define a graph that uses the connectivity map of the quantum system `ibmq_casablanca`.

In [5]:
num_qubits = 7

# adjacency matrix for `ibmq_casablanca`
adjmat = [
    [0, 1, 0, 0, 0, 0, 0], 
    [1, 0, 1, 1, 0, 0, 0], 
    [0, 1, 0, 0, 0, 0, 0], 
    [0, 1, 0, 0, 0, 1, 0], 
    [0, 0, 0, 0, 0, 1, 0], 
    [0, 0, 0, 1, 1, 0, 1], 
    [0, 0, 0, 0, 0, 1, 0]]

### Your code goes here

How would you prepare a graph state with high fidelity? In the example below, we create it using Qiskit's circuit library at the gate level. You may explore other methods for creating the graph states, including by using pulse-level techniques or accounting for the errors in the quantum system.

In [6]:
def create_graph_state(props):
    
    ### YOUR CODE GOES HERE -- START
    
    sgs = solve_graph_state(["min_target_spectator", "runtime"], gl_in=props, t1t2_in=props)
    sgs.build_model()
    qc = sgs.solve()

    ### YOUR CODE GOES HERE -- END
    
    return qc

In [7]:
# the graph state can be created using Qiskit's circuit library
state_circuit = create_graph_state(properties)
state_circuit.draw()

┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐     »
q_0: ┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ X ├┤ H ├─────»
     ├───┤├───┤├───┤└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└─┬─┘└───┘     »
q_1: ┤ I ├┤ X ├┤ H ├──────────────────────────────────────────■────────────»
     ├───┤└─┬─┘├───┤┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
q_2: ┤ I ├──┼──┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├»
     ├───┤  │  ├───┤├───┤├───┤├───┤├───┤├───┤├───┤├───┤├───┤├───┤├───┤├───┤»
q_3: ┤ H ├──■──┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ H ├»
     ├───┤     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
q_4: ┤ H ├──■──────────────────────────────────────────────────────────────»
     ├───┤┌─┴─┐┌───┐                                                       »
q_5: ┤ I ├┤ X ├┤ H ├───────────────────────────────────────────────────────»
     ├───┤├───┤├───┤┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
q_6: ┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├»
     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«                                                                           
«q_0: ──────────────────────────────────────────────────────────────────────
«                                                                           
«q_1: ─────────────────────────────────────────────────────────■────────────
«     ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌─┴─┐┌───┐     
«q_2: ┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ X ├┤ H ├─────
«     ├───┤├───┤└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘     
«q_3: ┤ X ├┤ H ├────────────────────────────────────────────────────────────
«     └─┬─┘└───┘                                                            
«q_4: ──┼───────────────────────────────────────────────────────────────────
«       │                                                                   
«q_5: ──■───────────────────────────────────────────────────────────■───────
«     ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌─┴─┐┌───┐
«q_6: ┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ X ├┤ H ├
«     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘

## Creating and measuring stabilizers

We begin by defining functions to create stabilizer measurement circuits, and then appending them onto the circuit used to create the graph states.

In [8]:
def compute_stabilizer_group(circuit):
    """Compute the stabilizer group for stabilizer circuit."""
    state = qi.Statevector.from_instruction(circuit)
    labels = []
    for i in qi.pauli_basis(state.num_qubits):
        val = round(qi.state_fidelity(i.to_matrix()[0], state, validate=False))
        if val != 0:
            label = i.to_labels()[0]
            if val == 1:
                label = '+' + label
            else:
                label = '-' + label
            labels.append(label)
    return labels

def stabilizer_coeff_pauli(stabilizer):
    """Return the 1 or -1 coeff and Pauli label."""
    coeff = 1
    pauli = coeff
    if stabilizer[0] == '-':
        coeff = -1
    if stabilizer[0] in ['+', '-']:
        pauli = stabilizer[1:]
    else:
        pauli = stabilizer
    return coeff, pauli

def stabilizer_measure_circuit(stabilizer, initial_circuit=None):
    """Return a stabilizer measurement circuits.
    
    Args:
        stabilizer (str): a stabilizer string
        initial_circuit (QuantumCircuit): Optional, the initial circuit.
    
    Returns:
        QuantumCircuit: the circuit with stabilizer measurements.
    """
    _, pauli = stabilizer_coeff_pauli(stabilizer)
    if initial_circuit is None:
        circ = QuantumCircuit(len(pauli))
    else:
        circ = initial_circuit.copy()
    for i, s in enumerate(reversed(pauli)):
        if s == 'X':
            circ.h(i)
        if s == 'Y':
            circ.sdg(i)
            circ.h(i)
    circ.measure_all()
    return circ

In [9]:
## Compute the stabilizers for this graph state
generators = qi.Clifford(state_circuit).stabilizer.pauli.to_labels()
stabilizers = compute_stabilizer_group(state_circuit)
print('Stabilizers:', stabilizers)
print('Generators:', generators)

Stabilizers: ['+IIIIIII', '+IIIIIZX', '+IIIIXIX', '+IIIIXZI', '-IIIZYXY', '+IIIZYYZ', '+IIIZZXZ', '+IIIZZYY', '+IIXXIIX', '+IIXXIZI', '+IIXXXII', '+IIXXXZX', '-IIXYYXZ', '-IIXYYYY', '-IIXYZXY', '+IIXYZYZ', '+IZIXIIX', '+IZIXIZI', '+IZIXXII', '+IZIXXZX', '-IZIYYXZ', '-IZIYYYY', '-IZIYZXY', '+IZIYZYZ', '+IZXIIII', '+IZXIIZX', '+IZXIXIX', '+IZXIXZI', '-IZXZYXY', '+IZXZYYZ', '+IZXZZXZ', '+IZXZZYY', '+XIIXIIX', '+XIIXIZI', '+XIIXXII', '+XIIXXZX', '-XIIYYXZ', '-XIIYYYY', '-XIIYZXY', '+XIIYZYZ', '+XIXIIII', '+XIXIIZX', '+XIXIXIX', '+XIXIXZI', '-XIXZYXY', '+XIXZYYZ', '+XIXZZXZ', '+XIXZZYY', '+XZIIIII', '+XZIIIZX', '+XZIIXIX', '+XZIIXZI', '-XZIZYXY', '+XZIZYYZ', '+XZIZZXZ', '+XZIZZYY', '+XZXXIIX', '+XZXXIZI', '+XZXXXII', '+XZXXXZX', '-XZXYYXZ', '-XZXYYYY', '-XZXYZXY', '+XZXYZYZ', '+YXYIYXY', '-YXYIYYZ', '-YXYIZXZ', '-YXYIZYY', '-YXYZIII', '-YXYZIZX', '-YXYZXIX', '-YXYZXZI', '-YXZXYXZ', '-YXZXYYY', '-YXZXZXY', '+YXZXZYZ', '-YXZYIIX', '-YXZYIZI', '-YXZYXII', '-YXZYXZX', '-YYYXYXZ', '-YYYXYYY', '-

In [10]:
## Append the stabilizer measurements to the graph state circuit 
stabilizer_circuits = [stabilizer_measure_circuit(stab, state_circuit)
                       for stab in stabilizers]

stabilizer_circuits[0].draw()

┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐     »
   q_0: ┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ X ├┤ H ├─────»
        ├───┤├───┤├───┤└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└─┬─┘└───┘     »
   q_1: ┤ I ├┤ X ├┤ H ├──────────────────────────────────────────■────────────»
        ├───┤└─┬─┘├───┤┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
   q_2: ┤ I ├──┼──┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├»
        ├───┤  │  ├───┤├───┤├───┤├───┤├───┤├───┤├───┤├───┤├───┤├───┤├───┤├───┤»
   q_3: ┤ H ├──■──┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ H ├»
        ├───┤     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
   q_4: ┤ H ├──■──────────────────────────────────────────────────────────────»
        ├───┤┌─┴─┐┌───┐                                                       »
   q_5: ┤ I ├┤ X ├┤ H ├───────────────────────────────────────────────────────»
        ├───┤├───┤├───┤┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
   q_6: ┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├»
        └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
meas: 7/══════════════════════════════════════════════════════════════════════»
                                                                              »
«                                                                              »
«   q_0: ──────────────────────────────────────────────────────────────────────»
«                                                                              »
«   q_1: ─────────────────────────────────────────────────────────■────────────»
«        ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌─┴─┐┌───┐     »
«   q_2: ┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ X ├┤ H ├─────»
«        ├───┤├───┤└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘     »
«   q_3: ┤ X ├┤ H ├────────────────────────────────────────────────────────────»
«        └─┬─┘└───┘                                                            »
«   q_4: ──┼───────────────────────────────────────────────────────────────────»
«          │                                                                   »
«   q_5: ──■───────────────────────────────────────────────────────────■───────»
«        ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌─┴─┐┌───┐»
«   q_6: ┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ X ├┤ H ├»
«        └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«meas: 7/══════════════════════════════════════════════════════════════════════»
«                                                                              »
«         ░ ┌─┐                  
«   q_0: ─░─┤M├──────────────────
«         ░ └╥┘┌─┐               
«   q_1: ─░──╫─┤M├───────────────
«         ░  ║ └╥┘┌─┐            
«   q_2: ─░──╫──╫─┤M├────────────
«         ░  ║  ║ └╥┘┌─┐         
«   q_3: ─░──╫──╫──╫─┤M├─────────
«         ░  ║  ║  ║ └╥┘┌─┐      
«   q_4: ─░──╫──╫──╫──╫─┤M├──────
«         ░  ║  ║  ║  ║ └╥┘┌─┐   
«   q_5: ─░──╫──╫──╫──╫──╫─┤M├───
«         ░  ║  ║  ║  ║  ║ └╥┘┌─┐
«   q_6: ─░──╫──╫──╫──╫──╫──╫─┤M├
«         ░  ║  ║  ║  ║  ║  ║ └╥┘
«meas: 7/════╩══╩══╩══╩══╩══╩══╩═
«            0  1  2  3  4  5  6

## Measurement mitigation

Noisy measurements impact our ability to accurately measure the state fidelity. For our default example we calibrate our measurements for the CTMP method using states with two-qubit excitations.

In [11]:
def _calibration_circuit(num_qubits: int, label: str, gates='x', delay: int = 0) -> QuantumCircuit:
    """Return a calibration circuit.

    This is an N-qubit circuit where N is the length of the label.
    The circuit consists of HZH-gates on qubits with label bits equal to 1,
    and measurements of all qubits.
    """
    circ = QuantumCircuit(num_qubits, name='meas_mit_cal_' + label)
    for i, val in enumerate(reversed(label)):
        if val == '1':
            if gates == 'z':
                circ.x(i)
            elif gates == 'x':
                circ.h(i)
                circ.z(i)
                circ.barrier()

            elif gates == 'y':
                circ.h(i)
                circ.s(i)
                circ.z(i)
            else:
                raise ValueError("wrong parameter, only x, y, z is supported")
        else:
            if gates == 'z':
                pass
            elif gates == 'x':
                circ.h(i)
            elif gates == 'y':
                circ.h(i)
                circ.s(i)
            else:
                raise ValueError("wrong parameter, only x, y, z is supported")
    one_indices =[i for i, v in enumerate(reversed(label)) if v == '1']
    if len(one_indices) > 0:
        for j in range(delay):
            circ.id(one_indices)
            circ.barrier()
    else:
        for j in range(delay):
            circ.id(range(7))
            circ.barrier()

    if gates == 'z':
        pass
    elif gates == 'x':
        circ.h(range(num_qubits))
    elif gates == 'y':
        circ.sdg(range(num_qubits))
        circ.h(range(num_qubits))

    circ.measure_all()
    return circ

In [12]:
labels = ['0000000', '0000011', '0000101', 
          '0001001', '0001010', '0001100', 
          '0010001', '0010010', '0010100', '0011000', 
          '0100001', '0100010', '0100100', '0101000', '0110000', 
          '1000001', '1000010', '1000100', '1001000', '1010000', '1100000', 
          '1111111']

three_cnot_delay_in_id_gates = 33
meas_cal_circuits = [_calibration_circuit(7, l, gates='x', delay=three_cnot_delay_in_id_gates) for l in labels]
# this is just the original metadata returned by the call to mit.expval_meas_mitigator_circuits
#metadata = pickle.load(open("metadata.p",'rb'))
metadata = [{'experiment': 'meas_mit', 'cal': '0000000', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '0000011', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '0000101', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '0001001', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '0001010', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '0001100', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '0010001', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '0010010', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '0010100', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '0011000', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '0100001', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '0100010', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '0100100', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '0101000', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '0110000', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '1000001', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '1000010', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '1000100', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '1001000', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '1010000', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '1100000', 'method': 'CTMP'},
 {'experiment': 'meas_mit', 'cal': '1111111', 'method': 'CTMP'}]

#meas_cal_circuits, metadata = mit.expval_meas_mitigator_circuits(num_qubits, labels=labels)

In [13]:
#[meas_cal_circuits_full, state_labels] = mit.complete_meas_cal(range(num_qubits))

## Run the circuits

We will run the circuits on the `ibmq_casablanca` quantum system.

In order to debug more quickly and avoid queues, you may consider using a simulator backend modeled after the real quantum system. This will use the noise model of `ibmq_casablanca` to do simulations. You may uncomment the line below to do so. **Note that the fidelities of your graph states will generally be higher on the simulator, but the Open Science Prize is awarded for the best fidelities on the real quantum system.**

In [14]:
# backend = QasmSimulator.from_backend(provider.get_backend('ibmq_casablanca'))

We run the measurement calibration circuits in a separate job from the graph state circuits.  We repeat both 16 times and use the mean as the final value. 

In order to debug more quickly, you may consider reducing `reps` from 16 to 1. **Note that the final submissions will need to be executed with 16 repetitions.**

In [15]:
reps = 16

In [ ]:
all_jobs = []
all_jobs_mit = []

for ii in range(reps):

    # Run QPT on backend
    shots = 8192
    il = [0,1,2,3,4,5,6]
    
    job_mit_backend = execute(meas_cal_circuits, backend, shots=shots, initial_layout=il)
    job_backend = execute(stabilizer_circuits, backend, shots=shots, initial_layout=il)
    print('Job IDs ({}/{}): \n measurement calibration: {}\n stabilizer measurements: {}'.format(
        ii+1, reps, job_mit_backend.job_id(), job_backend.job_id()))

    all_jobs.append(job_backend)
    all_jobs_mit.append(job_mit_backend)

Job IDs (1/16): 
 measurement calibration: 304c1c47-fffa-4d44-b7d7-e4ab344e863f
 stabilizer measurements: 5c2b4bee-95ce-4ca8-b109-37652bf98b3a
Job IDs (2/16): 
 measurement calibration: bafa850f-eb45-49ca-aa77-ed851a394409
 stabilizer measurements: 666effba-2fde-4248-ba3e-b01b353bf3ba
Job IDs (3/16): 
 measurement calibration: 2d58c0b2-3a02-4173-aed5-1a54a2524309
 stabilizer measurements: 98f2d752-ea04-4f06-9fa6-72446faf462b
Job IDs (4/16): 
 measurement calibration: f2afb54f-7f67-4f3a-8190-b27b002b6353
 stabilizer measurements: abf270da-de60-4b46-ad14-ab6ea9f7cba2
Job IDs (5/16): 
 measurement calibration: f60787f4-6b20-4262-a2a8-a0b58cfad2c5
 stabilizer measurements: 405b523e-26d1-4180-b30b-1d9c98a72f3b
Job IDs (6/16): 
 measurement calibration: 426ed6a8-665d-4830-989b-bd0c21ab35f9
 stabilizer measurements: 69f0f872-2fbc-4c62-8323-c354471cc197
Job IDs (7/16): 
 measurement calibration: 563b7974-04cf-420e-83f6-7094120a4094
 stabilizer measurements: cc9d7ee4-9e82-42b8-ae68-63a10b4297e1

We can monitor the status of the jobs using Qiskit's job monitoring tools.

In [ ]:
for job in all_jobs:
    job_monitor(job)
    try:
        if job.error_message() is not None:
            print(job.error_message())
    except:
        pass

## Post-processing and computing fidelities

Once the jobs are completed, we can get the results back as follows.

In [ ]:
result_backend = []
result_mit_backend = []
for job in all_jobs:
    # Retrieve results (this may take a while depending on the queue)
    result_backend.append(job.result())
    
for job in all_jobs_mit:
    result_mit_backend.append(job.result())

Finally, we compute the fidelities of the graph states. You may consider creating your own method for error mitigation by updating the `stabilizer_expvals` function below. Here, we will use the default methods provided in Qiskit.

In [ ]:
def stabilizer_measure_diagonal(stabilizer):
    """Return the diagonal vector for a stabilizer measurement.
    
    Args:
        stabilizer (str): a stabilizer string
    
    Returns:
        np.ndarray: the diagonal for measurement in the stabilizer basis.
    """
    coeff, pauli = stabilizer_coeff_pauli(stabilizer)
    diag = np.array([1])
    for s in reversed(pauli):
        if s == 'I':
            tmp = np.array([1, 1])
        else:
            tmp = np.array([1, -1])
        diag = np.kron(tmp, diag)
    return coeff * diag
    
def stabilizer_fidelity(expvals, stddevs=None):
    """Compute stabilizer state fidelity from stabilizer expvals."""
    mean = np.mean(expvals)
    if stddevs is None:
        return mean
    stddev = np.sqrt(np.sum(stddevs ** 2))
    return mean, stddev

### Your code goes here

You may consider updating the function below to change how the measurement calibration circuits are used to compute the fidelity of the graph state.

In [ ]:
def stabilizer_expvals(result, stabilizers, meas_mitigator=None):
    """Compute expectation values from stabilizer measurement results."""

    ### YOUR CODE GOES HERE -- START
    
    expvals = []
    stddevs = []
    for i, stab in enumerate(stabilizers):
        expval, stddev = mit.expectation_value(
            result.get_counts(i),
            diagonal=stabilizer_measure_diagonal(stab),
            meas_mitigator=meas_mitigator)
        # mitigator 'overshoots' sometimes (expval > 1.0), since the fidelity is avg expvals, this can lead to fidelity > 1.0
        expval = 1.0 - np.abs(1.0 - expval)
        expvals.append(expval)
        stddevs.append(stddev)
    return np.array(expvals), np.array(stddevs)

    ### YOUR CODE GOES HERE -- END

In [ ]:
## Mitigate the stabilizer expectation values 
F_nomit_backend = []
F_mit_backend = []

for ii in range(reps):
    # Unmitigated Expectation Values
    expvals_nomit_b, stddevs_nomit_b = stabilizer_expvals(
        result_backend[ii], stabilizers)
  
    # Fit measurement error mitigators
    mitigator_backend = mit.ExpvalMeasMitigatorFitter(result_mit_backend[ii], metadata).fit()

    # Measurement error mitigated expectation values
    expvals_mit_b, stddevs_mit_b = stabilizer_expvals(
        result_backend[ii], stabilizers, meas_mitigator=mitigator_backend)
    
    # save the fidelities for this iteration
    F_nomit_backend.append(stabilizer_fidelity(expvals_nomit_b, stddevs_nomit_b)[0])
    F_mit_backend.append(stabilizer_fidelity(expvals_mit_b, stddevs_mit_b)[0])

Report the fidelity estimates.

In [ ]:
## The final results

print('Graph-state fidelity estimates')
print('\nNo mitigation')
print('F({}) = {:.3f} \u00B1 {:.3f}'.format(
    properties.backend_name, np.mean(F_nomit_backend), np.std(F_nomit_backend)))

print('\nCTMP error mitigation')
print('F({}) = {:.3f} \u00B1 {:.3f}'.format(
    properties.backend_name, np.mean(F_mit_backend), np.std(F_mit_backend)))

## Qiskit version

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table